In [2]:
# Standard Python Imports

import numpy as np
import pandas as pd
import csv
#from numba import njit
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import matplotlib
import statsmodels.formula.api as sm
import statsmodels.formula.api as smf
import math
from scipy.stats import norm
from random import choices
from numpy import matlib
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components
from scipy.stats import mode
from scipy.sparse import identity
from scipy.sparse.linalg import inv as spinv
from scipy.sparse.linalg import spsolve as spsolve
from numpy.linalg import eig

inv, ax, randint = np.linalg.inv, np.newaxis, np.random.randint
repmat, rand = np.matlib.repmat, np.random.rand

## Read in data

In [29]:
datapath = "/Users/nadialucas/Documents/fabfour/pset1/va-data.csv"
df = pd.read_csv(datapath)
df.head()



,year,id_school,id_grade,id_teacher,id_class,id_student,id_cohort,m_education,hh_income,sd_indv,score_std,lag_score_std
0,2000,1,3,1,3,6,6,8,13566.4960,0.090079,0.370097,NaN
1,2000,1,3,1,3,3906,6,12,4036.5371,0.090079,-2.011367,NaN
2,2000,1,3,1,3,4556,6,21,15218.5520,0.090079,0.732662,NaN
3,2000,1,3,1,3,5206,6,18,11004.5960,0.090079,-0.251898,NaN
4,2000,1,3,1,3,10406,6,11,7115.0620,0.090079,-1.278978,NaN


## Estimate teacher VA from student scores (Appendix A)

Drift is accounted for by permitting coefficients on score data to vary non-parametrically according to the distance between observed score and forecast year

### Step 1: Residualization of test scores

Residualize student scores with respect to controls by running OLS with teacher fixed effects

In [30]:
# teacher on student scores OLS
# what should controls, X_it be?
# (i) cubic polynomial in prior-year scores, interacted with student grade level
# (ii) class and school-year means of all other individual covariates
# (iii) class size and class type indicators
# (iv) grade and year dummies
regstring = 'score_std ~ class_m_education + class_hh_income + school_m_education + school_hh_income + class_size'
regstring = regstring + '+ C(id_teacher) + C(id_grade) + C(year)'

### (i) Interact those bad boys ###
for i in range(3,9):
    for j in range(1,4):
        df['grade_'+str(i)+'std'+str(j)] = 0
        df.loc[(df['id_grade'] == i), 'grade_'+str(i)+'std'+str(j)] = df['lag_score_std']**j
        regstring = regstring + ' + ' + 'grade_'+str(i)+'std'+str(j)
        
# help with identifiers
#def make_identifier(df):
 #   str_id = df.apply(lambda x: '_'.join(map(str, x)), axis=1)
 #   return pd.factorize(str_id)[0]

#df['class_school_id'] = make_identifier(df[['id_class','id_school', 'id_grade', 'year']])
#df['school_year_id'] = make_identifier(df[['id_school', 'year']])

# generate (ii)
df['class_m_education'] = df.groupby(['id_class', 'id_school', 'id_grade', 'year'])['m_education'].transform(lambda x: x.mean())
df['class_hh_income'] = df.groupby(['id_class', 'id_school', 'id_grade', 'year'])['hh_income'].transform(lambda x: x.mean())

df['school_m_education'] = df.groupby(['id_school', 'year'])['m_education'].transform(lambda x: x.mean())
df['school_hh_income'] = df.groupby(['id_school', 'year'])['hh_income'].transform(lambda x: x.mean())

# generate (iii) - what is a class type indicator? I dont think we have one 
df['class_size'] = df.groupby(['id_class', 'id_school', 'id_grade', 'year'])['year'].transform(lambda x: x.count())

regstring = regstring + ' + class_m_education + class_hh_income + school_m_education + school_hh_income + class_size'

pd.set_option('display.max_columns', None)
print(df.head())
print(regstring)

# I'm doing something dumb here but I can't figure out what
results = smf.ols(formula = regstring, data = df).fit()
# this is wrong, we wanna also remove the teacher fixed effects and idk how...
K = length(coef(results))
df['res'] = results.resid

   year  id_school  id_grade  id_teacher  id_class  id_student  id_cohort  \
0  2000          1         3           1         3           6          6   
1  2000          1         3           1         3        3906          6   
2  2000          1         3           1         3        4556          6   
3  2000          1         3           1         3        5206          6   
4  2000          1         3           1         3       10406          6   

   m_education   hh_income   sd_indv  score_std  lag_score_std  grade_3std1  \
0            8  13566.4960  0.090079   0.370097            NaN          NaN   
1           12   4036.5371  0.090079  -2.011367            NaN          NaN   
2           21  15218.5520  0.090079   0.732662            NaN          NaN   
3           18  11004.5960  0.090079  -0.251898            NaN          NaN   
4           11   7115.0620  0.090079  -1.278978            NaN          NaN   

   grade_3std2  grade_3std3  grade_4std1  grade_4std2  grade_4

PatsyError: Error evaluating factor: TypeError: 'DataFrame' object is not callable
    score_std ~ class_m_education + class_hh_income + school_m_education + school_hh_income + class_size+ C(id_teacher) + C(id_grade) + C(year) + grade_3std1 + grade_3std2 + grade_3std3 + grade_4std1 + grade_4std2 + grade_4std3 + grade_5std1 + grade_5std2 + grade_5std3 + grade_6std1 + grade_6std2 + grade_6std3 + grade_7std1 + grade_7std2 + grade_7std3 + grade_8std1 + grade_8std2 + grade_8std3 + class_m_education + class_hh_income + school_m_education + school_hh_income + class_size
                                                                                                                                        ^^^^^^^

### Step 2: Estimation of variance component

1. Estimate individual level variance of residual test scores

sigma_eps = MSE * (N-1)/(N-K-C+1)

where MSE is variance of within-classroom deviations of A_it, N is total number of students, C is total number of classrooms, and K is number of control variables in the X_it vector

2. Then construct precision-weighted averages of classroom-average scores within a teacher-year

weight_ct = 1/(sigma_theta + (sigma_eps/n_ct))

where sigma_theta is estimate of class-level variance and n_ct is num student sin classroom

3. Now estimate sigma_A0 which is covariance of test scores of adjacent classrooms in each teacher-year cell (weighting each pair by sum of students taught)

4. Then estimate sigma_As which is covariance between mean scores across years within teachers

In [42]:
# get N, K, C
N = df.count()
C = df.groupby('id_class').count()
K = 11 # not correct but a placeholder for now

# for now just a placeholder
df['res'] = df['score_std']
# compute total variance
sigma_A = df.loc[:,'res'].var() * ((N-1)/(N-K))

# compute within class variance
df['class_mean'] = df.groupby(['id_school', 'year', 'id_class', 'id_teacher'])['res'].transform(lambda x: x.mean())
df['dev_from_mean'] = df.groupby(['id_school', 'year', 'id_class', 'id_teacher'])['res'].transform(lambda x: x-x.mean())
sigma_eps = df.loc[:,'dev_from_mean'].var() * ((N-1)/(N-K-C+1))

# compute class-level variance
# create class-level dataset
df_class_means = df[['id_school', 'year', 'id_grade', 'id_teacher', 'id_cohort', 'id_class', 'class_mean', 'class_size']]
df_class = df_class_means.drop_duplicates()
df_class.head(20)
# ok tbh not sure how to do this but going to bed now


# create teacher-level dataset

,id_school,year,id_grade,id_teacher,id_cohort,id_class,class_mean,class_size
0,1,2000,3,1,6,3,-0.142960,26
26,1,2000,3,2,6,4,0.232649,25
51,1,2000,3,3,6,2,-0.095379,27
78,1,2000,3,4,6,1,-0.255499,32
110,1,2000,4,5,5,3,0.034354,30
140,1,2000,4,6,5,1,0.056037,28
168,1,2000,4,7,5,2,-0.401731,28
196,1,2000,4,8,5,4,0.285647,30
226,1,2000,5,9,4,3,-0.502755,25
251,1,2000,5,10,4,2,-0.058030,32


### Step 3: Construction of VA estimates

1. Construct bar(A_j_-t) = vector of teacher-year-mean scores used to predict teacher j's VA in year t. N_jt is the length of this vector

2. Construct BLP of teacher quality in year t as 

mu_jt = (inv(Sigma_Ajt)\*gamma_jt)'\*bar(A_j_-t)

where gamma_jt is N_jt x 1, Sigma_Ajt is N_jt x N_jt - see appendix for what each element is

and we are done